Import modules

In [0]:
import tensorflow as tf
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

데이터 불러오기, 결측치 제거, 변수 선택

In [113]:
# 데이터 불러오기

data = pd.read_csv("hyunse_df.csv",encoding='utf-8')
print("전체 데이터 개수 : ",len(data))

# 결측치 제거

data = data.dropna(subset=['distance','record',
                           'code','lane','age','jockey_w','dandivi','yeondivi',
                           'cure_in_1m','weight_diff', 'raw_weight','weight_added','jockey',
                           'prev1_velo','prev2_velo','sex1','sex2'])

print("결측치 제거 후 개수 : ",len(data))


# distance별로 데이터 나누기

gb_distance = data.groupby(['distance'])
gb_distance=[gb_distance.get_group(x) for x in gb_distance.groups]

# distance list
distance_list = list(set(data['distance']))
distance_list.sort()

for i in range(len(gb_distance)):
  print(distance_list[i] ,"m 거리에서 데이터 개수 : ",len(gb_distance[i]))


# 데이터 가장 많은 Dataframe 선택  

data=gb_distance[3]


# X,y 데이터 나누기

y = data['record']
X = data.loc[:, ['code','lane','age','jockey_w','dandivi','yeondivi',
                 'cure_in_1m','weight_diff', 'raw_weight','weight_added','jockey',
                 'prev1_velo','prev2_velo','sex1','sex2']]

# 
X = X.as_matrix()
y = y.as_matrix()


전체 데이터 개수 :  58749
결측치 제거 후 개수 :  56577
1000.0 m 거리에서 데이터 개수 :  6395
1100.0 m 거리에서 데이터 개수 :  482
1200.0 m 거리에서 데이터 개수 :  13576
1300.0 m 거리에서 데이터 개수 :  12912
1400.0 m 거리에서 데이터 개수 :  9928
1700.0 m 거리에서 데이터 개수 :  5612
1800.0 m 거리에서 데이터 개수 :  5686
1900.0 m 거리에서 데이터 개수 :  1082
2000.0 m 거리에서 데이터 개수 :  849
2300.0 m 거리에서 데이터 개수 :  55


In [0]:
# 설정하는 부분

learning_rate = 0.01
training_epochs = 500
batch_size = 128


In [118]:
# Train , Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Normalize 

X_mean = []
X_std = []

for i in range(X_train.shape[1]):
  X_mean.append(np.mean(X_train[:,i]))
  X_std.append(np.std(X_train[:,i]))
  print(i+1,"번째 변수의 X_mean:",'{:.3f}'.format(X_mean[i]),", X_std:",'{:.3f}'.format(X_std[i]))
  X_train[:,i] = (X_train[:,i]-X_mean[i])/X_std[i]
  X_test[:,i] = (X_test[:,i]-X_mean[i])/X_std[i]

  
train_size = len(X_train)
print("train_size : ",train_size)

size_x = len(X_train[0])
print("size_x : ",size_x)



1 번째 변수의 X_mean: 32590.31544589511 , X_std: 3343.5875592698426
2 번째 변수의 X_mean: 6.167293649037398 , X_std: 3.387999531863979
3 번째 변수의 X_mean: 3.5299845098473113 , X_std: 0.8577371459184414
4 번째 변수의 X_mean: 53.85837574684665 , X_std: 1.7033453460757029
5 번째 변수의 X_mean: 38.13348085859704 , X_std: 51.66977820893637
6 번째 변수의 X_mean: 6.36862137641071 , X_std: 7.244394564254882
7 번째 변수의 X_mean: 0.24231024562956407 , X_std: 0.42848102699244967
8 번째 변수의 X_mean: 0.16651914140296525 , X_std: 6.957624627818564
9 번째 변수의 X_mean: 467.8983182119938 , X_std: 27.255492419956507
10 번째 변수의 X_mean: 0.16651914140296525 , X_std: 6.957624627818564
11 번째 변수의 X_mean: 80429.9752157557 , X_std: 123.22868295832517
12 번째 변수의 X_mean: 15.518547687404984 , X_std: 0.3684910096329752
13 번째 변수의 X_mean: 15.512970643059926 , X_std: 0.37544994262357967
14 번째 변수의 X_mean: 0.32208453197610093 , X_std: 0.4672751718611175
15 번째 변수의 X_mean: 0.45872980747953085 , X_std: 0.498293860296636
train_size :  9038
size_x :  15


In [119]:
print(X_train.shape, X_test.shape)

(9038, 15) (3874, 15)


In [0]:
# Tensorflow

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, size_x])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)
phase = tf.placeholder(tf.bool, name='phase')

# First Layer

W1 = tf.get_variable(shape=[size_x, 128], name='weight1', initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([128]))
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)
layer1 = tf.contrib.layers.batch_norm(layer1, center=True, scale=True, 
                                          is_training=phase)

# Second Layer

W2 = tf.get_variable(shape=[128, 64], name='weight2', initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
layer2 = tf.nn.dropout(layer2, keep_prob=keep_prob)
layer2 = tf.contrib.layers.batch_norm(layer2, center=True, scale=True, 
                                          is_training=phase)
# Third Layer

W3 = tf.get_variable(shape=[64, 32], name='weight3', initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([32]))
layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)
layer3 = tf.nn.dropout(layer3, keep_prob=keep_prob)
layer3 = tf.contrib.layers.batch_norm(layer3, center=True, scale=True, 
                                          is_training=phase)
# Predict

W4 = tf.get_variable(shape=[32, 1], name='weight4', initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([1]))
hypothesis = tf.matmul(layer3, W4) + b4
hypothesis = tf.contrib.layers.batch_norm(hypothesis, center=True, scale=True, 
                                          is_training=phase)

# Cost 계산

cost = tf.reduce_mean(tf.square(hypothesis - Y))
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)



In [121]:
# 계산 시작

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_size / batch_size)

    for i in range(total_batch):
        # batch_size로 X,y 개수 나눠주기
        batch_xs = X_train[i*batch_size:(i+1)*batch_size]
        batch_ys = y_train[i*batch_size:(i+1)*batch_size].reshape(batch_size,1)
        
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:0.7,phase:True}
        c, _ = sess.run([cost, train], feed_dict=feed_dict)
        avg_cost += c/total_batch
        
    # 20 epoch마다 cost
    
    if epoch % 20==0:
      print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
      


Epoch: 0001 cost = 7039.829303850
Epoch: 0021 cost = 4960.786725725
Epoch: 0041 cost = 3339.680991908
Epoch: 0061 cost = 2094.030064174
Epoch: 0081 cost = 1180.608921596
Epoch: 0101 cost = 565.134487915
Epoch: 0121 cost = 210.198478263
Epoch: 0141 cost = 56.332824217
Epoch: 0161 cost = 22.727941261
Epoch: 0181 cost = 18.942418596
Epoch: 0201 cost = 19.351690456
Epoch: 0221 cost = 20.107229246
Epoch: 0241 cost = 20.291400705
Epoch: 0261 cost = 19.787217127
Epoch: 0281 cost = 20.009228631
Epoch: 0301 cost = 19.424526814
Epoch: 0321 cost = 19.109680802
Epoch: 0341 cost = 18.650255864
Epoch: 0361 cost = 18.761173657
Epoch: 0381 cost = 18.001524033
Epoch: 0401 cost = 18.920152255
Epoch: 0421 cost = 18.770695571
Epoch: 0441 cost = 18.232621329
Epoch: 0461 cost = 18.679840108
Epoch: 0481 cost = 18.821387843


In [122]:
# Test set 계산

y_test = y_test.reshape(-1,1)
mse = tf.reduce_mean(tf.square(hypothesis - Y))**0.5
print('Cost of test set:', sess.run(mse, feed_dict={
      X: X_test, Y: y_test, keep_prob:1, phase:False}))

Cost of test set: 30016.006
